# 프로젝트: 가위바위보 분류기 만들기

## 데이터 준비하기

### 데이터 만들기

분류기를 만들기 위해서는 학습시킬 데이터들이 많이 필요한데요.


여기에서는 일단 노트북에 있는 웹캠만 있으면 빠르고 쉽게 많은 양의 이미지데이터를 확보할 수 있는

구글~~갓~~의 teachable machine 사이트를 이용해 데이터를 만들어보겠습니다.

<https://teachablemachine.withgoogle.com/>

![](~/Pictures/Screenshot from 2021-01-07 15-44-58.png)

    1. 원하는 클래스명을 입력
    
    2. *Hold to Record* 버튼을 꾹 누르고 찍고싶은 형태를 열정적으로 돌려가면서 찍습니다.~~후방주의~~
    
    3. 메뉴를 클릭하여 Download samples를 클릭


가위, 바위, 보 사진을 충분히 찍어서 각각의 폴더를 만들어서 저장을 하면 데이터 확보는 끝났습니다.


하지만 이 이미지 파일들은 아직 아무런 가공도 되지 않은 상태이기 때문에 우리가 사용할 수 있게 데이터를 수정해야합니다.

이번에는 PIL라이브러리를 사용하기 위해 28\*28 픽셀 이미지를 사용할 예정이기 때문에 사진들의 크기를 조절하는 작업을 해줄 겁니다.

### 데이터 Resize하기

In [1]:
# PIL 라이브러리 설치
!pip install pillow   

from PIL import Image
import os, glob

# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
# 저는 학습용 이미지 폴더를 따로 만들었습니다.
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train/scissor"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  


target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train/rock"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("바위 이미지 resize 완료!")

# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train/paper"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("보 이미지 resize 완료!")

이미지 디렉토리 경로:  /home/aiffel/aiffel/rock_scissor_paper_train/scissor
가위 이미지 resize 완료!
이미지 디렉토리 경로:  /home/aiffel/aiffel/rock_scissor_paper_train/rock
바위 이미지 resize 완료!
이미지 디렉토리 경로:  /home/aiffel/aiffel/rock_scissor_paper_train/paper
보 이미지 resize 완료!


이제 확보한 이미지 데이터들의 가공이 끝났습니다.

*이미지 데이터를 학습시킬때에는 가로축과 세로축의 픽셀값을 일정하게 맞춰줘야하기 때문에*

위와 같은 작업은 필수적입니다.


이제 분류기가 데이터를 읽을 수 있도록 해주는 *load_data()*라는 함수를 선언할 건데요.

우리는 이 함수를 통해 rock_scissor_paper_train폴더 안에 있는 이미지 개수의 총합, 크기, 색상 정보를

담을 *행렬*을 만들겁니다.


저는 보다 정확한 분류기를 만들기 위해 AIFFEL 학우들의 데이터를 함께 모아서 15300개의 이미지파일을 사용했습니다.

### 데이터 불러오기

In [2]:
import numpy as np

def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=15300   # 가위바위보 이미지 개수 총합
    img_size=28
    color=3                # 흑백사진은 1, 유색사진은 RGB 3가지 색상을 사용하므로 3을 넣어줍니다.
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0          # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1          # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=2          # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.") # 총 개수 확인
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_train"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0                           # 입력은 0~1 사이의 값으로 정규화
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

# 행렬의 형태 확인
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 15000 입니다.
x_train shape: (15300, 28, 28, 3)
y_train shape: (15300,)


여기까지 했으면 훈련 데이터의 준비는 완료했고 이제 학습 모델을 설계해야 될 차례입니다.

## 딥러닝 네트워크 설계하기

이제 딥러닝 네트워크를 만들 차례인데요.

여기에서는 tensorflow의 keras를 사용해 설계해 보겠습니다.

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model=keras.models.Sequential()

# 가로세로 28 pixel의 유색이미지 파일에 대해 32개의 이미지 특징을 고려함
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3))) 
model.add(keras.layers.MaxPool2D(2,2))

# 첫번째 이미지 특징 저장한 이후 64개의 이미지 특징을 고려함
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Flatten())

# 분류기의 알고리즘 복잡도 설정
model.add(keras.layers.Dense(64, activation='relu'))

# 총 클래스의 개수 가위,바위,보 = 3가지
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                102464    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1

## 딥러닝 네트워크 학습시키기

이제 설계된 분류기를 학습시켜 볼 차례입니다.

In [4]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 분류기의 학습횟수를 설정
model.fit(x_train_reshaped, y_train, epochs=15)

Epoch 1/15
479/479 [==============================] - 5s 10ms/step - loss: 0.7859 - accuracy: 0.6310
Epoch 2/15
479/479 [==============================] - 1s 1ms/step - loss: 0.4155 - accuracy: 0.8440
Epoch 3/15
479/479 [==============================] - 1s 1ms/step - loss: 0.2527 - accuracy: 0.9127
Epoch 4/15
479/479 [==============================] - 1s 1ms/step - loss: 0.1583 - accuracy: 0.9478
Epoch 5/15
479/479 [==============================] - 1s 1ms/step - loss: 0.1064 - accuracy: 0.9679
Epoch 6/15
479/479 [==============================] - 1s 1ms/step - loss: 0.0801 - accuracy: 0.9758
Epoch 7/15
479/479 [==============================] - 1s 1ms/step - loss: 0.0518 - accuracy: 0.9848
Epoch 8/15
479/479 [==============================] - 1s 1ms/step - loss: 0.0401 - accuracy: 0.9880
Epoch 9/15
479/479 [==============================] - 1s 1ms/step - loss: 0.0453 - accuracy: 0.9865
Epoch 10/15
479/479 [==============================] - 1s 1ms/step - loss: 0.0222 - accuracy: 0.994

## 테스트하기

### 테스트 데이터 준비하기

저는 제가 확보한 데이터는 모두 학습하는데 사용했기 때문에

아까와 동일한 방법으로 300장의 이미지 파일을 다시 만들고

그것들을 *test용 폴더*에 넣었습니다.


이제 아까와 동일한 방법으로 테스트 데이터를 만들 차례입니다.

In [5]:
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=300   # 가위바위보 이미지 개수 총합
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper_test" # 다른 폴더 주의
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 3)

print("x_test shape: {}".format(x_test.shape))
print("y_ttest shape: {}".format(y_test.shape))

학습데이터(x_test)의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)
x_test shape: (300, 28, 28, 3)
y_ttest shape: (300,)


### 테스트

드디어 마지막 차례입니다.

이 결과로 우리가 열심히 만든 분류기가 좋은지 나쁜지 결정이 납니다.

In [6]:
test_loss, test_accuracy = model.evaluate(x_test_reshaped,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 2s - loss: 2.1160 - accuracy: 0.7167
test_loss: 2.116013288497925 
test_accuracy: 0.7166666388511658


저는 정확도가 0.73이 나왔습니다.

1이 나왔으면 했지만 더 양질의 데이터가 많이 필요할 것 같네요.